In [4]:
import sys, os

# προσθέτουμε το src/ ως entry για imports
package_path = os.path.abspath('../src')
if package_path not in sys.path:
    sys.path.insert(0, package_path)

from symbolic_regression.methods.gp import GP
from symbolic_regression.utils.pysr_utils import nrmse_loss

In [6]:
pysr_params = {
    "populations": 1,
    "population_size": 20,
    "niterations": 50,
    "binary_operators": ["+", "-", "*"],
    "unary_operators": ["sqrt", "inv(x) = 1/x"],
    "extra_sympy_mappings": {"inv": lambda x: 1/x},
    "verbosity": 0
}

In [7]:
gp = GP(
    loss_function=nrmse_loss ,
    record_interval=5,
    pysr_params=pysr_params
)

In [ ]:
# prepei na kanw imptort data

TypeError: 'BaseMethod' object is not callable